<a href="https://colab.research.google.com/github/aaa7353351-max/Object-Detection-YOLO/blob/main/YOLOv8_%EA%B8%B0%EB%B0%98_%EB%8F%84%EB%A1%9C_%ED%91%9C%EC%A7%80%ED%8C%90_%ED%83%90%EC%A7%80_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

• 데이터셋 링크
 https://www.kaggle.com/datasets/andrewmvd/road-sign-detection

In [ ]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download andrewmvd/road-sign-detection
# zip파일 다운로드

In [5]:
from pathlib import Path

In [8]:
root = Path('./road-sign_dataset')
Path('./road-sign_dataset/train').mkdir(parents=True, exist_ok=True)
Path('./road-sign_dataset/val').mkdir(parents=True, exist_ok=True)
Path('./road-sign_dataset/test').mkdir(parents=True, exist_ok=True)

In [7]:
for path1 in ('images', 'labels'):
    for path2 in ('train', 'val', 'test'):
        new_path = root / 'sign' / path1 / path2
        new_path.mkdir(parents=True, exist_ok=True)

In [9]:
!unzip -q road-sign-detection.zip

In [11]:
base_dir = '/content/Road_Sign_dataset/Sign'
xml_dir = '/content/annotations'
img_dir = '/content/images'

In [12]:
import os

In [13]:
dest_dirs = {
    'train': {
        'images': os.path.join(base_dir,'images/train'),
        'labels': os.path.join(base_dir, 'labels/train')
    },
    'val': {
        'images': os.path.join(base_dir, 'images/val'),
        'labels': os.path.join(base_dir, 'labels/val')
    },
    'test': {
        'images': os.path.join(base_dir, 'images/test'),
        'labels': os.path.join(base_dir, 'labels/test')
    }
}

In [17]:
import glob
import random

In [18]:
xml_files = glob.glob(os.path.join(xml_dir, '*.xml'))
file_ids = [Path(f).stem for f in xml_files]

random.seed(2026)
random.shuffle(file_ids)

In [19]:
total_files = len(file_ids)
train_idx = int(total_files * 0.6)
val_idx = int(total_files * 0.2)

In [20]:
train_ids = file_ids[:train_idx]
val_ids = file_ids[train_idx : train_idx + val_idx]
test_ids = file_ids[train_idx + val_idx:]

In [21]:
print(f"Total files: {total_files}")
print(f"Train: {len(train_ids)}, Val: {len(val_ids)}, Test: {len(test_ids)}")


Total files: 877
Train: 526, Val: 175, Test: 176


In [22]:
import xml.etree.ElementTree as ET

In [23]:
classes = set()

In [24]:
xml_files = glob.glob(os.path.join(xml_dir, '*.xml'))

In [26]:
for xml_file in xml_files:
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        #  태그 내의  태그 값을 찾음
        for obj in root.findall('object'):
            name = obj.find('name').text
            classes.add(name)
    except Exception as e:
        print(f"Error parsing {xml_file}: {e}")

In [27]:
sorted_classes = sorted(list(classes))

In [28]:
output_file = 'voc.names'
with open(output_file, 'w') as f:
    for cls in sorted_classes:
        f.write(cls + '\n')

print(f"\n[완료] '{output_file}' 파일이 생성되었습니다.")
print(f"발견된 클래스 개수: {len(sorted_classes)}")
print(f"클래스 목록: {sorted_classes}")


[완료] 'voc.names' 파일이 생성되었습니다.
발견된 클래스 개수: 4
클래스 목록: ['crosswalk', 'speedlimit', 'stop', 'trafficlight']


In [30]:
import shutil

In [32]:
def copy_dataset(ids, split_name):
    print(f"\nProcessing {split_name} data...")

    dest_img_path = dest_dirs[split_name]['images']
    dest_lbl_path = dest_dirs[split_name]['labels']

    os.makedirs(dest_img_path, exist_ok=True)
    os.makedirs(dest_lbl_path, exist_ok=True)

    for file_id in ids:
        src_xml = os.path.join(xml_dir, file_id + '.xml')
        shutil.move(src_xml, os.path.join(dest_lbl_path, file_id + '.xml'))

        found_image = False
        for ext in ['.jpg', '.png', '.jpeg', '.JPG']:
            src_img = os.path.join(img_dir, file_id + ext)
            if os.path.exists(src_img):
                shutil.move(src_img, os.path.join(dest_img_path, file_id + ext))
                found_image = True
                break

        if not found_image:
            print(f"Warning: Image for {file_id} not found.")

copy_dataset(train_ids, 'train')
copy_dataset(val_ids, 'val')
copy_dataset(test_ids, 'test')



Processing train data...


FileNotFoundError: [Errno 2] No such file or directory: '/content/annotations/road32.xml'

In [ ]:
!python example.py \
    --datasets VOC \
    --img_path /content/Road_Sign_dataset/Sign/images/train \
    --label /content/Road_Sign_dataset/Sign/labels/train \
    --convert_output_path /content/Road_Sign_dataset/Sign/labels/train \
    --img_type '.png' \
    --manifest_path /content/Road_Sign_dataset/Sign \
    --cls_list_file ./voc.names

In [ ]:
!python example.py \
    --datasets VOC \
    --img_path /content/Road_Sign_dataset/Sign/images/val \
    --label /content/Road_Sign_dataset/Sign/labels/val \
    --convert_output_path /content/Road_Sign_dataset/Sign/labels/val \
    --img_type '.png' \
    --manifest_path /content/Road_Sign_dataset/Sign \
    --cls_list_file ./voc.names

In [ ]:
!python example.py \
    --datasets VOC \
    --img_path /content/Road_Sign_dataset/Sign/images/test \
    --label /content/Road_Sign_dataset/Sign/labels/test \
    --convert_output_path /content/Road_Sign_dataset/Sign/labels/test \
    --img_type '.png' \
    --manifest_path /content/Road_Sign_dataset/Sign \
    --cls_list_file ./voc.names


In [ ]:
%cd /content/

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8s.pt')

In [ ]:
# yaml 파일 생성
import yaml

# 1. voc.names에서 클래스 이름 읽어오기
with open('/content/Road_Sign_dataset/convert2Yolo/voc.names', 'r') as f:
    # 빈 줄 제거하고 이름만 리스트로 만듦
    classes = [line.strip() for line in f.readlines() if line.strip()]

# 2. YOLO 학습을 위한 YAML 데이터 정의
# path: 데이터셋의 루트 경로 (절대 경로)
# train, val, test: path 기준 상대 경로
data = {
    'path': '/content/Road_Sign_dataset/Sign',
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'names': {i: name for i, name in enumerate(classes)}  # {0: 'car', 1: 'bus'...} 형태
}

# 3. yaml 파일로 저장
yaml_path = '/content/Road_Sign_dataset/Sign/sign_data.yaml'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f, sort_keys=False)

print(f"설정 파일이 생성되었습니다: {yaml_path}")
print("내용 확인:")
print(data)

In [ ]:
results = model.train(
    data='/content/Road_Sign_dataset/Sign/sign_data.yaml',
    epochs=10,
    batch=32,
    imgsz=640,
    device=0,
    workers=2,
    name='custom_s'
)

In [ ]:
# 학습된 모델 로드
model = YOLO("/content/runs/detect/custom_s/weights/best.pt")

# 검증 수행
results = model.val(
    data="/content/Road_Sign_dataset/Sign/sign_data.yaml",  # 데이터셋 설정
    imgsz=640,               # 이미지 크기
    iou=0.5,                 # IoU 임계값
    batch=32,                # 배치 크기
    device=0,                # GPU 사용
    workers=2,               # 데이터 로드 시 병렬 처리할 워커 수
    half=True,               # FP16 연산 활성화 (속도 향상)
    split="val"
)

print(results)

In [ ]:
# 학습된 모델 로드
model = YOLO("/content/runs/detect/custom_s/weights/best.pt")

# 테스트 수행
results = model.val(
    data="/content/Road_Sign_dataset/Sign/sign_data.yaml",  # 데이터셋 설정
    imgsz=640,               # 이미지 크기
    iou=0.5,                 # IoU 임계값
    batch=32,                # 배치 크기
    device=0,                # GPU 사용
    workers=2,               # 데이터 로드 시 병렬 처리할 워커 수
    half=True,               # FP16 연산 활성화 (속도 향상)
    split="test"
)

print(results)


In [ ]:
import pandas as pd

# --- 1. 전체 종합 점수 (Highlight) ---
print("\n" + "="*40)
print(f"   🏆 최종 테스트 성능 요약 (Test Set)")
print("="*40)
print(f" • mAP@50-95 (종합 점수) : {results.box.map:.4f}")  # 0~1 사이, 높을수록 좋음
print(f" • mAP@50    (탐지 능력) : {results.box.map50:.4f}") # IoU 0.5 기준 점수
print(f" • Precision (정밀도)    : {results.box.mp:.4f}")   # 오탐지(False Positive)가 적은지
print(f" • Recall    (재현율)    : {results.box.mr:.4f}")   # 놓친 것(False Negative)이 적은지
print("="*40 + "\n")


# --- 2. 클래스별 상세 점수 (DataFrame 활용) ---
print("🔍 [클래스별 상세 성능 (mAP@50-95 기준)]")

# 클래스 이름과 해당 클래스의 mAP 점수 가져오기
class_ids = results.names.keys()
class_names = [results.names[i] for i in class_ids]
class_maps = results.box.maps  # 각 클래스별 mAP50-95 배열

df = pd.DataFrame({
    'Class Name': class_names,
    'mAP': class_maps
})

# 점수가 높은 순서대로 정렬해서 출력
df_sorted = df.sort_values(by='mAP', ascending=False).reset_index(drop=True)

# 표 출력
print(df_sorted.to_string(formatters={'mAP': '{:.4f}'.format}))


In [ ]:
model = YOLO("/content/runs/detect/custom_s/weights/best.pt")

results = model.predict(
    source="/content/Road_Sign_dataset/Sign/images/custom",  # 테스트 이미지 폴더
    imgsz=640,           # 입력 이미지 크기
    conf=0.25,           # 신뢰도(Confidence) 임계값
    device=0,            # GPU 사용 (CPU 사용 시 "cpu")
    save=True,           # 탐지 결과 저장
    save_txt=True,       # 탐지 결과를 txt 형식으로 저장 (YOLO 포맷)
    save_conf=True       # 탐지된 객체의 신뢰도 점수도 저장
)

print(results)


In [ ]:
for result in results:
    boxes = result.boxes  # 탐지된 박스 정보

    # 몇 개가 탐지되었는지 간략히 출력
    print(f"이미지: {result.path}")
    print(f"탐지된 객체 수: {len(boxes)}")
    print("-" * 20)

In [ ]:

# 1. 설정
n_cols = 4                     # 한 줄에 4개씩
n_images = len(results)        # 전체 이미지 개수
n_rows = math.ceil(n_images / n_cols)  # 필요한 행(줄) 개수 계산

# 2. 전체 캔버스 크기 설정 (가로 20, 세로는 행 개수에 비례해서 늘림)
plt.figure(figsize=(20, 5 * n_rows))

# 3. 반복문으로 하나씩 그리기
for i, r in enumerate(results):
    # 그리드 위치 지정 (행, 열, 몇 번째인지)
    plt.subplot(n_rows, n_cols, i + 1)

    # 이미지 변환 및 가져오기
    im_bgr = r.plot()
    im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)

    # 출력
    plt.imshow(im_rgb)
    plt.axis('off')             # 축 정보 제거
    plt.title(f"Result {i+1}")  # 제목 (선택)

plt.tight_layout() # 간격 자동 조절
plt.show()